In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Load data
data = pd.read_csv('DAT_MS_EURUSD_M1_202407.csv')

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Convert `Datetime` to datetime format
data['Datetime'] = pd.to_datetime(data['Datetime'], unit='ms')

# Sort data by datetime just in case
data = data.sort_values(by='Datetime')

# Use only the 'Close' prices for this example
prices = data['Close'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_prices = scaler.fit_transform(prices)

In [ ]:
look_back = 4 * 30 * 24 * 60  # 5 months of minute data (approx. 30 days per month)
predict_forward = 5  # Predict next 5 minutes

# Ensure there are enough data points
if len(scaled_prices) < look_back + predict_forward:
    raise ValueError("Not enough data points for the given look_back period.")

# Create dataset
def create_minute_dataset(dataset, look_back, predict_forward=5):
    X, Y = [], []
    for i in range(len(dataset) - look_back - predict_forward):
        time.sleep(3)
        a = dataset[i:(i + look_back), 0]
        X.append(a)
        Y.append(dataset[(i + look_back):(i + look_back + predict_forward), 0])
    return np.array(X), np.array(Y)

X, Y = create_minute_dataset(scaled_prices, look_back, predict_forward)

In [ ]:
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Build LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(look_back, 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(predict_forward))  # Predicting the next 5 minutes

# Compile and train model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X, Y, batch_size=1, epochs=10)

# Save the model
model.save('lstm_5months_model.h5')


In [3]:
import time